In [9]:
import numpy as np
import math

In [10]:
class HullWhite_grid:
    """
    af
    """
    def __init__(self, T, a, sigma, phi, r_0, N):
        """
        初期化する。
        Augs
            T: 最大の時間
            a: ドリフト
        """        
        # 入力を整理
        self.T = T
        self.a = a
        self.sigma = sigma
        self.phi = phi
        self.r_0 = r_0
        self.N = N  

        # 格子の縦・横の幅を設定
        self.delta_t = self.T / self.N
        self.delta_x = self.sigma * (3 * self.delta_t) ** (1 / 2)

        # 格子の端点を設定
        self.I = math.ceil(6 ** (1 / 2) / (3 * self.a * self.delta_t))
        
        # 諸々の格子を作成する
        self.X_matrix = self._make_X_matrix()
        self.r_matrix = self._make_r_matrix()
        
    def calc_discount_bond_price(self, payoff = 1):
        """
        backwardで割引債価格を求める
        Augs
            payoff: 満期における割引債価格のペイオフ(指定しなければ1)
        """
        discount_bond_matrix = np.zeros((self.I * 2 + 1, self.N))
        discount_bond_matrix[:, self.N - 1] = payoff

        for j in reversed(range(self.N - 1)):
            for i in range(-self.I, self.I + 1):
                if abs(i) > j:
                    # 不要な部分は計算しない
                    continue

                # 推移確率を計算
                prob = self._calc_prob(i)

                if abs(i) < self.I:
                    """
                    タイプA
                    """
                    discount_bond_matrix[self.I - i, j] += discount_bond_matrix[self.I - i - 1, j + 1] * prob["up"]
                    discount_bond_matrix[self.I - i, j] += discount_bond_matrix[self.I - i, j + 1] * prob["mid"]
                    discount_bond_matrix[self.I - i, j] += discount_bond_matrix[self.I - i + 1, j + 1] * prob["down"]

                elif i == self.I:
                    """
                    タイプC
                    """ 
                    # 格子上側
                    discount_bond_matrix[self.I - i, j] += discount_bond_matrix[self.I - i + 2, j + 1] * prob["down_2"]
                    discount_bond_matrix[self.I - i, j] += discount_bond_matrix[self.I - i + 1, j + 1] * prob["down"]
                    discount_bond_matrix[self.I - i, j] += discount_bond_matrix[self.I - i, j + 1] * prob["mid"]

                elif i == - self.I:
                    """
                    タイプB
                    """ 
                    # 格子下側
                    discount_bond_matrix[self.I - i, j] += discount_bond_matrix[self.I - i - 2, j + 1] * prob["up_2"]
                    discount_bond_matrix[self.I - i, j] += discount_bond_matrix[self.I - i - 1, j + 1] * prob["up"]
                    discount_bond_matrix[self.I - i, j] += discount_bond_matrix[self.I - i, j + 1] * prob["mid"]

                # 金利を掛ける
                discount_bond_matrix[I - i, j] *= math.exp(- self.r_matrix[I - i, j] * self.delta_t) 
        return discount_bond_matrix
    
    def _make_X_matrix(self):
        """
        Xの格子を作成する
        (I * 2 + 1) × Nの配列をベースに作成
        """
        X_matrix = np.zeros((self.I * 2 + 1, self.N))
        for j in range(self.N):
            # j：列番号
            for i in range(min([j + 1, self.I + 1])):
                # i: 中心線からの距離
                # 表からはみ出ないようにしつつ、値を代入
                X_matrix[self.I - i, j] = i * self.delta_x
                X_matrix[self.I + i, j] = - i * self.delta_x

        return X_matrix
    
    def _theta_function(self, t):
        theta = self.phi / self.a * (1 - math.exp(- self.a * t)) + self.r_0 * math.exp(- self.a * t)        
        return theta
    
    def _make_r_matrix(self):
        """
        スポットレートの格子を作る
        X_matrixをtheta(t)分シフトするだけ
        """
        r_matrix = self.X_matrix.copy()
                
        for j in range(self.N):
            # j：列番号
            r_matrix[self.I, j] += self._theta_function(self.delta_t * j)

            for i in range(1, min([j + 1, self.I + 1])):
                # i: 中心線からの距離
                # 表からはみ出ないようにしつつ、値を加算
                r_matrix[self.I - i, j] += self._theta_function(self.delta_t * j)
                r_matrix[self.I + i, j] += self._theta_function(self.delta_t * j)
        return r_matrix

    def _calc_prob(self, i):
        """
        推移確率を計算する
        """
        if abs(i) < I:
            """
            タイプA
            """
            # i: 中心線からの距離
            # 各点の確率を計算
            eta = (self.a * i * self.delta_x * self.delta_t) ** 2 + self.sigma ** 2 * self.delta_t
            up_prob = (eta - self.a * i * self.delta_x ** 2 * self.delta_t) / (2 * self.delta_x ** 2)
            mid_prob = 1 - eta / self.delta_x ** 2
            down_prob = (eta + self.a * i * self.delta_x ** 2 * self.delta_t) / (2 * self.delta_x ** 2)

            down_2_prob = 0
            up_2_prob = 0

        elif i == self.I:
            """
            タイプC
            """ 
            # 格子上側
            mid_prob = 7/6 + (self.a ** 2 * i ** 2 * self.delta_t ** 2 - 3 * self.a * i * self.delta_t) / 2
            down_prob = -1/3 - self.a ** 2 * i ** 2 * self.delta_t ** 2 + 2 * self.a * i * self.delta_t
            down_2_prob = 1/6 + (self.a ** 2 * i ** 2 * self.delta_t ** 2 - self.a * i * self.delta_t) / 2
            
            up_prob = 0
            up_2_prob = 0
            
        elif i == -self.I:
            """
            タイプB
            """ 
            # 格子下側
            up_2_prob = 1/6 + (self.a ** 2 * i ** 2 * self.delta_t ** 2 - self.a * abs(i) * self.delta_t) / 2
            up_prob = -1/3 - self.a ** 2 * i ** 2 * self.delta_t ** 2 + 2 * self.a * abs(i) * self.delta_t
            mid_prob = 7/6 + (self.a ** 2 * i ** 2 * self.delta_t ** 2 - 3 * self.a * abs(i) * self.delta_t) / 2

            down_prob = 0
            down_2_prob = 0
            
        return {"up_2": up_2_prob,
                "up": up_prob,
                "mid": mid_prob,
                "down": down_prob, 
                "down_2": down_2_prob}

Set parameters

In [11]:
T = 5
a = 0.2
sigma = 0.02
phi = 0.05
r_0 = 0.05

In [12]:
N = 5

In [13]:
delta_t = T / N
delta_x = sigma * (3 * delta_t) ** (1 / 2)

delta_t, delta_x

(1.0, 0.034641016151377546)

In [14]:
I = math.ceil((6 ** (1 / 2)) / (3 * a * delta_t))
I

5

In [15]:
(6 ** (1 / 2)) / (3 * a * delta_t)

4.0824829046386295

Set a Class

In [16]:
instance = HullWhite_grid(T, a, sigma, phi, r_0, N)

In [17]:
r = instance.r_matrix
X = instance.X_matrix

In [18]:
dis = instance.calc_discount_bond_price()

In [19]:
for i in range(5):
    print(instance._theta_function(i))

0.05
0.08625384938440364
0.11593599079287215
0.14023767278119473
0.1601342071765557
